In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install ipywidgets
# !pip install splink
# !jupyter nbextension enable --py widgetsnbextension

In [12]:
import splink.comparison_library as cl
from splink.internals.testing import comparison_vector_value, is_in_level
from splink import DuckDBAPI

db_api = DuckDBAPI()

comparison = cl.ExactMatch("first_name")

comparison_vector_value(comparison, {"first_name_l": "Robin", "first_name_r": "Robin"}, db_api)

{'comparison_vector_value': 1, 'label_for_charts': 'Exact match on first_name'}

In [61]:
def create_comparison_playground(column_name):

    comparison_types = [
        'ExactMatch', 'LevenshteinAtThresholds', 'JaroAtThresholds',
        'JaroWinklerAtThresholds', 'DamerauLevenshteinAtThresholds',
        'JaccardAtThresholds',
        'AbsoluteDateDifferenceAtThresholds', 'AbsoluteTimeDifferenceAtThresholds',
        'ArrayIntersectAtSizes', 'DateOfBirthComparison', 'DistanceFunctionAtThresholds',
        'DistanceInKMAtThresholds', 'EmailComparison', 'ForenameSurnameComparison',
        'NameComparison', 'PostcodeComparison'
    ]

    default_values = {
        'ExactMatch': ('john', 'jon'),
        'LevenshteinAtThresholds': ('smith', 'smyth'),
        'JaroAtThresholds': ('martha', 'matha'),
        'JaroWinklerAtThresholds': ('williams', 'willaims'),
        'DamerauLevenshteinAtThresholds': ('receive', 'recieve'),
        'CosineSimilarityAtThresholds': ('data science', 'science data'),
        'JaccardAtThresholds': ('0123456789', '012345678'),
        'AbsoluteDateDifferenceAtThresholds': ('2023-01-01', '2023-01-15'),
        'AbsoluteTimeDifferenceAtThresholds': ('12:00:00', '12:15:30'),
        'ArrayIntersectAtSizes': ('apple,banana,cherry', 'banana,cherry,date'),
        'DateOfBirthComparison': ('1990-05-15', '1990-05-16'),
        'DistanceFunctionAtThresholds': ('10', '15'),
        'DistanceInKMAtThresholds': ('51.5074,-0.1278', '51.5074,-0.1290'),
        'EmailComparison': ('john.doe@example.com', 'johndoe@example.com'),
        'ForenameSurnameComparison': ('John Doe', 'Jon Doe'),
        'NameComparison': ('Elizabeth Taylor', 'Elisabeth Taylor'),
        'PostcodeComparison': ('SW1A 1AA', 'SW1A 1AB')
    }

    db_api = DuckDBAPI()

    docstrings = {}
    for comp_type in comparison_types:
        class_obj = getattr(cl, comp_type)
        init_doc = getattr(class_obj.__init__, '__doc__', None)
        docstrings[comp_type] = init_doc if init_doc else class_obj.__doc__


    def get_comparison(comp_type):
        if comp_type == 'DateOfBirthComparison':
            return getattr(cl, comp_type)(column_name, input_is_string=True)
        elif comp_type in ['EmailComparison', 'ForenameSurnameComparison', 'NameComparison', 'PostcodeComparison', 'ArrayIntersectAtSizes']:
            return getattr(cl, comp_type)(column_name)
        else:
            return getattr(cl, comp_type)(column_name)

    def run_comparison(change):
        left_value = left_input.value
        right_value = right_input.value
        comparison = get_comparison(comparison_select.value)

        if comparison_select.value == 'ArrayIntersectAtSizes':
            left_value = left_value.split(',')
            right_value = right_value.split(',')

        output.clear_output()
        with output:
            display(Markdown("### Comparison levels:"))
            report_comparison_levels(comparison, left_value, right_value, column_name)
            docstring = docstrings.get(comparison_select.value, "No docstring available")

            # Process the docstring to remove indentation and render as markdown

            processed_docstring = "\n".join(line.strip() for line in docstring.split("\n"))
            display(Markdown("### Comparison Function Docstring:"))
            display(Markdown(processed_docstring))

    def on_comparison_change(change):
        new_value = change['new']
        left_value, right_value = default_values.get(new_value, ('', ''))

        # Temporarily unobserve the input widgets
        left_input.unobserve(run_comparison, names='value')
        right_input.unobserve(run_comparison, names='value')

        # Update the values
        left_input.value = left_value
        right_input.value = right_value

        # Re-observe the input widgets
        left_input.observe(run_comparison, names='value')
        right_input.observe(run_comparison, names='value')

        # Run the comparison once after updating both inputs
        run_comparison(None)


    comparison_select = widgets.Dropdown(
        options=comparison_types,
        value='ExactMatch',
        description='Comparison:',
    )
    left_input = widgets.Text(description=f"{column_name} Left:", value=default_values['ExactMatch'][0])
    right_input = widgets.Text(description=f"{column_name} Right:", value=default_values['ExactMatch'][1])
    output = widgets.Output()

    comparison_select.observe(on_comparison_change, names='value')
    for widget in (comparison_select, left_input, right_input):
        widget.observe(run_comparison, names='value')

    # Call run_comparison immediately to compute initial output
    run_comparison(None)

    return widgets.VBox([comparison_select, left_input, right_input, output])

playground = create_comparison_playground("column")
display(playground)

TypeError: AbsoluteTimeDifferenceAtThresholds.__init__() missing 3 required keyword-only arguments: 'input_is_string', 'metrics', and 'thresholds'

In [13]:
dir(cl)

['AbsoluteDateDifferenceAtThresholds',
 'AbsoluteTimeDifferenceAtThresholds',
 'ArrayIntersectAtSizes',
 'CosineSimilarityAtThresholds',
 'CustomComparison',
 'DamerauLevenshteinAtThresholds',
 'DateOfBirthComparison',
 'DistanceFunctionAtThresholds',
 'DistanceInKMAtThresholds',
 'EmailComparison',
 'ExactMatch',
 'ForenameSurnameComparison',
 'JaccardAtThresholds',
 'JaroAtThresholds',
 'JaroWinklerAtThresholds',
 'LevenshteinAtThresholds',
 'NameComparison',
 'PostcodeComparison',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [26]:
import splink.internals.comparison_library as cl
comparison_types = [
    'ExactMatch', 'LevenshteinAtThresholds', 'JaroAtThresholds',
    'JaroWinklerAtThresholds', 'DamerauLevenshteinAtThresholds',
    'CosineSimilarityAtThresholds', 'JaccardAtThresholds',
    'AbsoluteDateDifferenceAtThresholds', 'AbsoluteTimeDifferenceAtThresholds',
    'ArrayIntersectAtSizes', 'DateOfBirthComparison', 'DistanceFunctionAtThresholds',
    'DistanceInKMAtThresholds', 'EmailComparison', 'ForenameSurnameComparison',
    'NameComparison', 'PostcodeComparison'
]

default_values = {
    'ExactMatch': ('john', 'jon'),
    'LevenshteinAtThresholds': ('smith', 'smyth'),
    'JaroAtThresholds': ('martha', 'matha'),
    'JaroWinklerAtThresholds': ('williams', 'willaims'),
    'DamerauLevenshteinAtThresholds': ('receive', 'recieve'),
    'CosineSimilarityAtThresholds': ('data science', 'science data'),
    'JaccardAtThresholds': ('python programming', 'programming python'),
    'AbsoluteDateDifferenceAtThresholds': ('2023-01-01', '2023-01-15'),
    'AbsoluteTimeDifferenceAtThresholds': ('12:00:00', '12:15:30'),
    'ArrayIntersectAtSizes': ('apple,banana,cherry', 'banana,cherry,date'),
    'DateOfBirthComparison': ('1990-05-15', '1990-05-16'),
    'DistanceFunctionAtThresholds': ('10', '15'),
    'DistanceInKMAtThresholds': ('51.5074,-0.1278', '51.5074,-0.1290'),
    'EmailComparison': ('john.doe@example.com', 'johndoe@example.com'),
    'ForenameSurnameComparison': ('John Doe', 'Jon Doe'),
    'NameComparison': ('Elizabeth Taylor', 'Elisabeth Taylor'),
    'PostcodeComparison': ('SW1A 1AA', 'SW1A 1AB')
}

docstrings = {comp_type: getattr(cl, comp_type).__doc__ for comp_type in comparison_types}
cl.JaccardAtThresholds.


In [43]:
from splink.internals.testing import comparison_vector_value, is_in_level
from splink import DuckDBAPI
from IPython.display import display, Markdown

def report_comparison_levels(comparison, left_value, right_value, column_name):
    db_api = DuckDBAPI()
    levels = comparison.create_comparison_levels()

    messages = []
    matched_level = None
    total_levels = len(levels)
    for i, level in enumerate(levels):
        level_number = total_levels - i - 1
        label = level.create_label_for_charts()
        if matched_level is None and is_in_level(level, {f"{column_name}_l": left_value, f"{column_name}_r": right_value}, db_api):
            messages.append(f"**Level {level_number}: {label} (Matched)**")
            matched_level = level_number
        else:
            messages.append(f"Level {level_number}: {label}")
        messages.append("")  # Add an empty line after each level

    markdown_output = "\n".join(messages)
    display(Markdown(markdown_output))

In [48]:
cl.DateOfBirthComparison("a")

TypeError: DateOfBirthComparison.__init__() missing 1 required keyword-only argument: 'input_is_string'